In [44]:
import pandas as pd
import numpy as np
import os
import datetime

from sklearn.experimental import enable_hist_gradient_boosting, enable_iterative_imputer
from sklearn import datasets, linear_model, ensemble, gaussian_process, isotonic, kernel_ridge, neighbors, neural_network, svm, tree
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, max_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold,train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.compose import TransformedTargetRegressor, ColumnTransformer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, FunctionTransformer,  StandardScaler, LabelEncoder, LabelBinarizer, OrdinalEncoder, RobustScaler
from sklearn.inspection import permutation_importance
import joblib

import lightgbm  
import xgboost 
import catboost
# from keras.wrappers.scikit_learn import KerasRegressor
# from keras.models import Sequential
# from keras.layers import Dense
import optuna

# visualization lib
import matplotlib.pyplot as plt
import seaborn as sns
from rfpimp import permutation_importances, plot_importances, plot_corr_heatmap
# import shap
import eli5
from eli5.sklearn import PermutationImportance
from pandas_profiling import ProfileReport

from helper import absolute_score, show_pred, show_feature_importances, reduce_mem_usage

In [45]:
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

In [46]:
DATASET_NAME = 'on-trade'

DATA_PATH = f'./data/{DATASET_NAME}_data.csv'

MODEL_PATH = f'./models/{DATASET_NAME}_model.joblib'
SCALER_PATH = f'./models/{DATASET_NAME}_scaler.joblib'
IMPUTER_PATH = f'./models/{DATASET_NAME}_imputer.joblib'
SKU_ENCODER_PATH = f'./models/{DATASET_NAME}_sku_encoder.joblib'
BRAND_ENCODER_PATH = f'./models/{DATASET_NAME}_brand_encoder.joblib'

RESULT_CSV_PATH = f'./train_results/{DATASET_NAME}_train_result.csv'
RESULT_EXCEL_PATH = f'./train_results/{DATASET_NAME}_train_result.xlsx'

MAX_N_JOBS = 8

In [63]:
import pyodbc

server = 'czpls242\prod2' 
database = 'FC_Tool' 
username = 'Admin_FC_Tool' 
password = 'tw44aa2T'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password) 

query1='EXEC [dbo].[ML_Table_extension_data] @Country={0}, @FromYear={1}, @FromWeek={2}, @ToYear= {3}, @ToWeek={4}, @Market="{5}"'.format('NULL', 2016, 1, 2021, 15, DATASET_NAME.upper())


data = pd.read_sql(query1, cnxn)

data.head()

,SkuShort,Brand,Country,Year,Week,NumberWorkdays,PrevNumberWorkdays,NextNumberWorkdays,AvgTemp,AvgRain,AvgSun,IsLockdown,PdtHl,PrevWeekPdtHl1,BgtHl,SalesHl,PrevWeekSalesHl1,PrevWeekSalesHl2,PrevYearSalesHl1,PrevYearSalesHl2,OldPredSalesHl
0,01000,PILSNER URQUELL,CZ,2016,1,5,NaN,5.0,-1.164286,0.935714,1.014286,False,NaN,NaN,1901.850666,2057.73,NaN,NaN,NaN,NaN,1902.0890
1,01000,PILSNER URQUELL,SK,2016,1,4,NaN,4.0,-1.164286,0.935714,1.014286,False,NaN,NaN,227.634836,240.00,NaN,NaN,NaN,NaN,155.0000
2,01002,PILSNER URQUELL,CZ,2016,1,5,NaN,5.0,-1.164286,0.935714,1.014286,False,NaN,NaN,472.938882,394.50,NaN,NaN,NaN,NaN,375.6472
3,01002,PILSNER URQUELL,SK,2016,1,4,NaN,4.0,-1.164286,0.935714,1.014286,False,NaN,NaN,71.895601,49.80,NaN,NaN,NaN,NaN,29.9165
4,01003,PILSNER URQUELL,CZ,2016,1,5,NaN,5.0,-1.164286,0.935714,1.014286,False,NaN,NaN,3010.971784,2535.60,NaN,NaN,NaN,NaN,2311.0085


In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54652 entries, 0 to 54651
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SkuShort            54652 non-null  object 
 1   Brand               54652 non-null  object 
 2   Country             54652 non-null  object 
 3   Year                54652 non-null  int64  
 4   Week                54652 non-null  int64  
 5   NumberWorkdays      54652 non-null  int64  
 6   PrevNumberWorkdays  54512 non-null  float64
 7   NextNumberWorkdays  48793 non-null  float64
 8   AvgTemp             54652 non-null  float64
 9   AvgRain             54652 non-null  float64
 10  AvgSun              54652 non-null  float64
 11  IsLockdown          54652 non-null  bool   
 12  PdtHl               33476 non-null  float64
 13  PrevWeekPdtHl1      33087 non-null  float64
 14  BgtHl               47073 non-null  float64
 15  SalesHl             47202 non-null  float64
 16  Prev